# QUESTIONS 2-4:  Creating Database from Wikipedia Table


In [1]:
#IMPORT LIBRARIES 
import numpy as np
import pandas as pd
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium




In [2]:
Toronto_Website_URL=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Use Beautiful Soup to Evaluate HTML Code to Create Table

In [3]:
from bs4 import BeautifulSoup
Soup=BeautifulSoup(Toronto_Website_URL,'lxml')
#print(Soup.prettify())  Command was used to find Table Link

In [4]:
Toronto_Table=Soup.find('table',{'class':'wikitable sortable'})
#Toronto Table-used <td> and <tr> to separate rows/columns

### Commands to Create Database Table

In [5]:
A=[]
B=[]
C=[]

In [6]:

for row in Toronto_Table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells)==3: 
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))



In [7]:
df=pd.DataFrame(A, columns=['Postal Code'])
df['Borough']=B
df['Neighborhoods']=C


### Drop the "Not Assigned Boroughs"

In [8]:
#Drop the Not Assigned Boroughs
NotAssigned = df[df['Borough']== 'Not assigned'].index
df.drop(NotAssigned, inplace=True)


### Reset Index, Change Not Assigned Neighborhood, Group by Postal Code

In [9]:
df.reset_index(drop=True, inplace=True)
df.at[6,'Neighborhoods']="Queen's Park"

FinalDF=df.groupby('Postal Code').agg({'Borough':'first', 'Neighborhoods': ', '.join}).reset_index()
FinalDF = FinalDF.replace('\n','', regex=True)

### Final Database Table

In [10]:

FinalDF

,Postal Code,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Database Shape

In [11]:

FinalDF.shape

(103, 3)

# Add Latitutude and Longitude to Database Table

In [12]:
!pip install geocoder==1.5.0
import geocoder # import geocoder

D=[]
E=[]

#For Loop to obtain Postal Codes
for row in FinalDF:
    postal_code=FinalDF['Postal Code']

lat_lng_coords = None

i=0

#loop until you get the coordinates
for row in postal_code:
    lat_lng_coords = None
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[i]))
      lat_lng_coords = g.latlng
    D.append(lat_lng_coords[0])
    E.append(lat_lng_coords[1])
    i=i+1
    
    

    
    

In [13]:
FinalDF['Latitude']=D
FinalDF['Longitude']=E

### Final Database Table with Latitude and Longitude Added

In [14]:
FinalDF

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


# Exploring and Clustering Toronto Neighborhoods

### Creating a Map of Toronto Neighborhoods

In [15]:

while(lat_lng_coords is None):
      g = geocoder.arcgis('Toronto, Ontario')
      lat_lng_coords = g.latlng
    
TorontoLatitude=lat_lng_coords[0]
TorontoLongitude=lat_lng_coords[1]
TorontoMap = folium.Map(location=[TorontoLatitude, TorontoLongitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(FinalDF['Latitude'], FinalDF['Longitude'], FinalDF['Borough'], FinalDF['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMap)  
    
TorontoMap

### Map Of Bouroughs With Toronto

In [16]:

TorontoBoroughDF = FinalDF[FinalDF['Borough'].str.contains('Toronto')].reset_index(drop=True)

while(lat_lng_coords is None):
      g = geocoder.arcgis('Toronto, Ontario')
      lat_lng_coords = g.latlng
    
TorontoBoroughLat=lat_lng_coords[0]
TorontoBoroughLong=lat_lng_coords[1]
TorontoBoroughMap = folium.Map(location=[TorontoBoroughLat, TorontoBoroughLong], zoom_start=10)

for lat, lng, borough, neighborhood in zip(TorontoBoroughDF['Latitude'], TorontoBoroughDF['Longitude'], TorontoBoroughDF['Borough'], TorontoBoroughDF['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoBoroughMap)  
    
TorontoBoroughMap


In [32]:
# The code was removed by Watson Studio for sharing.

### Explore Toronto Borough Neighborhoods

#### First Explore First Neighborhood on List

In [18]:
import json
from pandas.io.json import json_normalize

TorontoBoroughDF.loc[0, 'Neighborhoods']

Neighborhood0Lat = TorontoBoroughDF.loc[0, 'Latitude'] 
Neighborhood0Long = TorontoBoroughDF.loc[0, 'Longitude'] 

Neighborhood0Name = TorontoBoroughDF.loc[0, 'Neighborhoods'] 

print('Latitude and longitude values of {} are {}, {}.'.format(Neighborhood0Name, 
                                                               Neighborhood0Lat, 
                                                               Neighborhood0Long))

LIMIT = 100
radius = 500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    Neighborhood0Lat,
    Neighborhood0Long,
    radius,
    LIMIT)

results = requests.get(url).json()
results

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Latitude and longitude values of The Beaches are 43.67653121600006, -79.29542499999997.


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


### Loop to Explore Toronto Borough Neighborhoods

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
    
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

TorontoBoroughVenues = getNearbyVenues(names=TorontoBoroughDF['Neighborhoods'],
                                   latitudes=TorontoBoroughDF['Latitude'],
                                   longitudes=TorontoBoroughDF['Longitude']
                                  )



In [20]:
print('There are {} uniques categories.'.format(len(TorontoBoroughVenues['Venue Category'].unique())))

There are 228 uniques categories.


### Analyzing the Neighborhoods in Toronto

In [21]:
# one hot encoding
Toronto_onehot = pd.get_dummies(TorontoBoroughVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = TorontoBoroughVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,...,0.00000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.015152,0.000000,0.000000,...,0.00000,0.015152,0.030303,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.000000,0.030000,0.000000,...,0.00000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,...,0.00000,0.000000,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.012346,0.000000,0.000000,...,0.00000,0.000000,0.049383,0.000000,0.000000,0.049383,0.012346,0.000000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.012346,0.000000,0.012346,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.00


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Breakfast Spot,Gastropub,American Restaurant,Restaurant,Bar,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Breakfast Spot,Bakery,Seafood Restaurant,Beer Bar,Hotel,Café,Steakhouse,Cheese Shop
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Restaurant,Furniture / Home Store,Bakery,Hotel,Vegetarian / Vegan Restaurant,Bar,Italian Restaurant,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Steakhouse,Hotel,Café,Sushi Restaurant,Italian Restaurant,Pizza Place,Gym,Seafood Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Gym / Fitness Center,Bar,Café,Restaurant,Park,Sandwich Place,Speakeasy,Pub


### CLUSTER TORONTO NEIGHBORHOODS

In [25]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoBoroughDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhoods')

Toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Health Food Store,Other Great Outdoors,Pub,Trail,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,0,Bus Line,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,0,Park,Burger Joint,Steakhouse,Board Shop,Italian Restaurant,Intersection,Sandwich Place,Movie Theater,Ice Cream Shop,Fruit & Vegetable Store
3,M4M,East Toronto,Studio District,43.660629,-79.334855,2,Diner,Brewery,Italian Restaurant,Gastropub,Grocery Store,Café,Bar,Coffee Shop,Pizza Place,Sushi Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,3,Construction & Landscaping,Bus Line,Swim School,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [26]:
# create map
map_clusters = folium.Map(location=[TorontoBoroughLat, TorontoBoroughLong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhoods'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Outdoor and Fitness Environment-Parks, Gyms, Health Food, Harbor

In [27]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Other Great Outdoors,Pub,Trail,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,East Toronto,0,Bus Line,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,East Toronto,0,Park,Burger Joint,Steakhouse,Board Shop,Italian Restaurant,Intersection,Sandwich Place,Movie Theater,Ice Cream Shop,Fruit & Vegetable Store
6,Central Toronto,0,Playground,Gym Pool,Park,Garden,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
8,Central Toronto,0,Convenience Store,Gym,Playground,Gym Pool,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
10,Downtown Toronto,0,Playground,Grocery Store,Park,Candy Store,Women's Store,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space
19,Downtown Toronto,0,Harbor / Marina,Thrift / Vintage Store,Park,Pier,Gym,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space
31,West Toronto,0,Park,Gym / Fitness Center,Bakery,Bus Line,Brazilian Restaurant,Supermarket,Middle Eastern Restaurant,Furniture / Home Store,Café,Bar


In [28]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,IT Services,Women's Store,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Area of Coffee Shops and Café's

In [29]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,2,Diner,Brewery,Italian Restaurant,Gastropub,Grocery Store,Café,Bar,Coffee Shop,Pizza Place,Sushi Restaurant
5,Central Toronto,2,Japanese Restaurant,Clothing Store,Food & Drink Shop,Gym,Park,Hotel,Breakfast Spot,Dog Run,Ethiopian Restaurant,Elementary School
7,Central Toronto,2,Dessert Shop,Café,Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Farmers Market,Fast Food Restaurant,Seafood Restaurant,Sushi Restaurant
9,Central Toronto,2,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
11,Downtown Toronto,2,Bakery,Coffee Shop,Market,Pizza Place,Café,Restaurant,Italian Restaurant,Pharmacy,Playground,Japanese Restaurant
12,Downtown Toronto,2,Coffee Shop,Restaurant,Japanese Restaurant,Sushi Restaurant,Gay Bar,Men's Store,Burger Joint,Fast Food Restaurant,Pub,Bubble Tea Shop
13,Downtown Toronto,2,Coffee Shop,Bakery,Theater,Boat or Ferry,Historic Site,Brewery,Hotel,Pet Store,Ice Cream Shop,Breakfast Spot
14,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Sporting Goods Shop,Burger Joint,Bakery,Ramen Restaurant
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Seafood Restaurant,Bakery,Cosmetics Shop,Clothing Store,Cocktail Bar,Beer Bar,Breakfast Spot
16,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Breakfast Spot,Bakery,Seafood Restaurant,Beer Bar,Hotel,Café,Steakhouse,Cheese Shop


In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Construction & Landscaping,Bus Line,Swim School,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


### Parks and Convience Foods

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Park,Restaurant,Women's Store,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
34,West Toronto,4,Convenience Store,Park,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


## Final Analysis

The area of Downtown Toronto is filled with coffee shops and cafe's and then has a wide variety of other business, including restaurants and retail stores.  Surrounding the downtown area, are several neighborhoods with parks, gyms and playgrounds once you get out of the main city.  This cluster can also be seen along the water line.